In [ ]:
# Step 1: Mount your Google Drive.
# You will be prompted to authorize access.
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Define your project's base path in Google Drive for cleanliness.
# IMPORTANT: Make sure you have created this folder in your Drive and uploaded your .zip file there.
# For example, create a folder called "Colab Notebooks" and inside it, a folder called "nemo_asr_project".
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/nemo_asr_project"

# Step 3: Install NVIDIA NeMo.
!pip install nemo_toolkit['all']

# Step 4: Unzip your prepared dataset from Drive into the local Colab session for faster access during training.
# We unzip it to /content/ for speed, but our source is safely in Drive.
!unzip "{PROJECT_PATH}/vn_asr_data_v1.zip" -d /content/data

In [8]:
import nemo.collections.asr as nemo_asr
import os

# --- Step 3: Load the Pre-trained Model ---

# We use the generic 'ASRModel' class. This is the "Universal Loader."
# It automatically detects if the model needs BPE (like Conformer) or not.
print(f"Attempting to load model 'stt_en_conformer_ctc_large' using the Universal Loader...")

asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name="stt_en_conformer_ctc_large")

print("Model loaded successfully!")

Attempting to load model 'stt_en_conformer_ctc_large' using the Universal Loader...
[NeMo I 2025-12-13 10:16:31 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.10.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_2.6.0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo
[NeMo I 2025-12-13 10:16:33 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2025-12-13 10:16:38 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2025-12-13 10:16:38 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket1/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket2/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket3/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket4/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket5/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket6/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket7/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    

[NeMo I 2025-12-13 10:16:38 nemo_logging:393] PADDING: 0
[NeMo I 2025-12-13 10:16:40 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.6.0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo.
Model loaded successfully!


In [16]:
import json
import random
import os

# --- Step 4: Verify Data Pipeline ---

manifest_path = '/content/data/train_manifest.json'

print(f"Reading manifest from: {manifest_path}")

# Read the manifest file
data_entries = []
with open(manifest_path, 'r') as f:
    for line in f:
        data_entries.append(json.loads(line))

print(f"Successfully loaded {len(data_entries)} samples from the training set.")

# Pick a random sample to inspect
sample = data_entries[0] # Let's look at the first one
audio_path = sample['audio_filepath']
duration = sample['duration']

print("\n=== Data Pipeline Check ===")
print(f"Audio File: {audio_path}")
print(f"Original Duration: {duration:.2f} seconds")
print(f"File Exists: {os.path.exists(audio_path)}")
print("===========================")

Reading manifest from: /content/data/train_manifest.json
Successfully loaded 7 samples from the training set.

=== Data Pipeline Check ===
Audio File: /Users/nguyenhuyvu/nemo-vietnamese-asr/audio/qZuxop5xj_E.wav
Original Duration: 1287.48 seconds
File Exists: False


In [19]:
import librosa
import soundfile as sf
import os
import json

# --- Step 5: End-to-End Inference Test (Robust API Check) ---

print("Preparing audio for inference...")

# 1. GET DATA FROM MANIFEST
# We reuse the 'sample' loaded in Step 4
original_path = sample['audio_filepath']
filename = os.path.basename(original_path) # Extracts 'qZuxop5xj_E.wav'

# 2. CONSTRUCT CORRECT COLAB PATH
colab_audio_path = os.path.join('/content/data/audio', filename)

if not os.path.exists(colab_audio_path):
    raise FileNotFoundError(f"Could not find audio at: {colab_audio_path}")

# 3. LOAD & CROP (First 30s)
y, sr = librosa.load(colab_audio_path, sr=16000, duration=30.0)
temp_path = '/content/temp_30s.wav'
sf.write(temp_path, y, sr)
print(f"Created temporary 30s inference clip: {temp_path}")

# 4. RUN INFERENCE (With API Auto-Fix)
print("Running NVIDIA NeMo Transcribe...")
asr_model.eval()

# Let's inspect what the model expects (for your own knowledge)
print("\n--- Debug: Method Signature ---")
try:
    import inspect
    print(inspect.signature(asr_model.transcribe))
except:
    pass
print("-------------------------------\n")

transcriptions = []

try:
    # Attempt 1: The Standard Way (Keyword Argument)
    print("Attempting with 'paths2audio_files'...")
    transcriptions = asr_model.transcribe(paths2audio_files=[temp_path])
except TypeError:
    try:
        # Attempt 2: Positional Argument (Likely Fix)
        print("Attempting with positional arguments...")
        transcriptions = asr_model.transcribe([temp_path])
    except TypeError:
        # Attempt 3: New 'audio' keyword (Possible 2.0 change)
        print("Attempting with 'audio' keyword...")
        transcriptions = asr_model.transcribe(audio=[temp_path])

# 5. DISPLAY RESULTS
print("\n" + "="*40)
print(f"🤖 MODEL: {model_name_ngc}")
print(f"📂 INPUT: {filename} (First 30s)")
print("-" * 40)
print("🎙️ TRANSCRIPTION:")
if transcriptions:
    print(f"'{transcriptions[0]}'")
else:
    print("ERROR: Could not run inference. Check logs.")
print("="*40)

[NeMo W 2025-12-13 10:30:45 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2025-12-13 10:30:45 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)


Preparing audio for inference...
Created temporary 30s inference clip: /content/temp_30s.wav
Running NVIDIA NeMo Transcribe...

--- Debug: Method Signature ---
(audio: Union[str, List[str], torch.Tensor, numpy.ndarray, torch.utils.data.dataloader.DataLoader], batch_size: int = 4, return_hypotheses: bool = False, num_workers: int = 0, channel_selector: Union[int, Iterable[int], str, NoneType] = None, augmentor: omegaconf.dictconfig.DictConfig = None, verbose: bool = True, timestamps: Optional[bool] = None, override_config: Optional[nemo.collections.asr.parts.mixins.transcription.TranscribeConfig] = None) -> Union[List[str], List[nemo.collections.asr.parts.utils.rnnt_utils.Hypothesis], Tuple[List[str]], Tuple[List[nemo.collections.asr.parts.utils.rnnt_utils.Hypothesis]]]
-------------------------------

Attempting with 'paths2audio_files'...
Attempting with positional arguments...


Transcribing: 1it [00:01,  1.96s/it]


🤖 MODEL: stt_en_conformer_ctc_large
📂 INPUT: qZuxop5xj_E.wav (First 30s)
----------------------------------------
🎙️ TRANSCRIPTION:
'Hypothesis(score=tensor(-112.7178), y_sequence=tensor([128, 128, 128, 128,  66, 128, 128, 128,   8, 128, 128,  44, 128, 128,
          8, 128,   3, 128, 128, 128, 128, 128, 128, 115, 128, 128, 128, 128,
        128,  25, 128, 128,  47,   8,  18,  30, 128, 128,  31, 128, 128, 128,
        128,  64, 128,  45, 128,  20, 128,   1,  30,  15,   4, 128, 128, 128,
         50,   1, 128,  28, 128,  22, 128,  20,  12, 128, 128,   1, 128,  93,
        128,   5, 128,   9,   6, 128, 128, 128, 128, 128, 128, 128,  63, 128,
        128,   6, 128, 128,   1, 128, 128, 128, 128, 128, 106, 128,  28, 128,
         31, 128, 128, 128, 128, 128,  85, 128, 128,   3, 128,  56, 128, 128,
          3,  35, 128,   4,  42, 128,   1,  14,  28, 128,  22,   1,  11, 128,
        128,  14, 128,  67, 128,   4,  47, 128, 128, 128, 128,  23, 128, 128,
        128,   5, 128,   8, 128, 128,  

In [23]:
import json
import os
import librosa
import soundfile as sf
import jiwer

# --- Step 6: Manual Evaluation Loop (Final Fix) ---

print("Starting Manual Evaluation on Test Set...")

test_manifest_path = '/content/data/test_manifest.json'

# 1. Read the Test Manifest
with open(test_manifest_path, 'r') as f:
    test_samples = [json.loads(line) for line in f]

print(f"Found {len(test_samples)} samples in test set.")

# 2. Evaluation Loop
asr_model.eval()

total_wer = 0
count = 0

for sample in test_samples:
    original_path = sample['audio_filepath']
    reference_text = sample['text']
    filename = os.path.basename(original_path)

    colab_path = os.path.join('/content/data/audio', filename)

    if not os.path.exists(colab_path):
        continue

    try:
        # Crop to 30s
        y, sr = librosa.load(colab_path, sr=16000, duration=30.0)
        temp_path = f"/content/temp_eval_{count}.wav"
        sf.write(temp_path, y, sr)

        # Transcribe
        try:
            # Try keyword arg first
            preds = asr_model.transcribe(paths2audio_files=[temp_path])
        except TypeError:
            # Fallback to positional
            preds = asr_model.transcribe([temp_path])

        # --- THE FIX IS HERE ---
        # Check if output is a string or an object
        hypothesis = preds[0]
        if not isinstance(hypothesis, str):
            # It's a Hypothesis object, extract the text
            hypothesis = hypothesis.text

        # Calculate Metrics
        wer = jiwer.wer(reference_text, hypothesis)
        total_wer += wer
        count += 1

        print(f"\n--- Sample {count} ---")
        print(f"Ref: {reference_text[:50]}...")
        print(f"Hyp: {hypothesis[:50]}...")
        print(f"WER: {wer:.2f}")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

if count > 0:
    avg_wer = total_wer / count
    print("\n" + "="*40)
    print(f"Evaluation Complete. Average WER: {avg_wer:.2f}")
    print("="*40)

[NeMo W 2025-12-13 10:35:47 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2025-12-13 10:35:47 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)


Starting Manual Evaluation on Test Set...
Found 2 samples in test set.


Transcribing: 1it [00:00,  5.20it/s]
[NeMo W 2025-12-13 10:35:48 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2025-12-13 10:35:48 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)



--- Sample 1 ---
Ref: chào bạn cảm ơn bạn đã nhấn nghe giang ngơi radio ...
Hyp: ta bak coming back do onion n ang the radio no mei...
WER: 1.00


Transcribing: 1it [00:00,  7.15it/s]


--- Sample 2 ---
Ref: chào bạn cảm ơn bạn đã nhấn nghe ra hơi radio nơi ...
Hyp: tbak coming back don your nas on the radio no momi...
WER: 1.00

Evaluation Complete. Average WER: 1.00


In [22]:
# Step 7: Save your final model directly to your Google Drive for persistence.
import os

model_save_path = os.path.join(PROJECT_PATH, "vietnamese_asr_v1.nemo")
print(f"Saving the final model to: {model_save_path}")
asr_model.save_to(model_save_path)
print("Model saved successfully to Google Drive!")

Saving the final model to: /content/drive/MyDrive/Colab Notebooks/nemo_asr_project/vietnamese_asr_v1.nemo
Model saved successfully to Google Drive!
